In [3]:
import pandas as pd
import datetime as dt
from prophet import Prophet
import numpy as np
from matplotlib import pyplot


%matplotlib inline

In [4]:
adidas_path = 'Resources/Adidas US Sales Datasets.xlsx'
adidas_df = pd.read_excel(adidas_path,
                          index_col='Unnamed: 0',
                          parse_dates=True)

In [5]:
adidas_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9648 entries, NaT to NaT
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Retailer          9648 non-null   object        
 1   Retailer ID       9648 non-null   int64         
 2   Invoice Date      9648 non-null   datetime64[ns]
 3   Region            9648 non-null   object        
 4   State             9648 non-null   object        
 5   City              9648 non-null   object        
 6   Product           9648 non-null   object        
 7   Price per Unit    9648 non-null   float64       
 8   Units Sold        9648 non-null   int64         
 9   Total Sales       9648 non-null   float64       
 10  Operating Profit  9648 non-null   float64       
 11  Operating Margin  9648 non-null   float64       
 12  Sales Method      9648 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(2), object(6)
memory usage: 1.0+ M

In [6]:
df = adidas_df.drop(['Retailer', 'Retailer ID', 'Total Sales', 'Operating Profit', 'Sales Method'], axis=1).dropna()
print(df)



    Invoice Date     Region          State        City  \
NaT   2020-01-01  Northeast       New York    New York   
NaT   2020-01-02  Northeast       New York    New York   
NaT   2020-01-03  Northeast       New York    New York   
NaT   2020-01-04  Northeast       New York    New York   
NaT   2020-01-05  Northeast       New York    New York   
..           ...        ...            ...         ...   
NaT   2021-01-24  Northeast  New Hampshire  Manchester   
NaT   2021-01-24  Northeast  New Hampshire  Manchester   
NaT   2021-02-22  Northeast  New Hampshire  Manchester   
NaT   2021-02-22  Northeast  New Hampshire  Manchester   
NaT   2021-02-22  Northeast  New Hampshire  Manchester   

                       Product  Price per Unit  Units Sold  Operating Margin  
NaT      Men's Street Footwear            50.0        1200              0.50  
NaT    Men's Athletic Footwear            50.0        1000              0.30  
NaT    Women's Street Footwear            40.0        1000        

In [7]:
df.set_index('Invoice Date', inplace=True)
print(df)

                 Region          State        City                    Product  \
Invoice Date                                                                    
2020-01-01    Northeast       New York    New York      Men's Street Footwear   
2020-01-02    Northeast       New York    New York    Men's Athletic Footwear   
2020-01-03    Northeast       New York    New York    Women's Street Footwear   
2020-01-04    Northeast       New York    New York  Women's Athletic Footwear   
2020-01-05    Northeast       New York    New York              Men's Apparel   
...                 ...            ...         ...                        ...   
2021-01-24    Northeast  New Hampshire  Manchester              Men's Apparel   
2021-01-24    Northeast  New Hampshire  Manchester            Women's Apparel   
2021-02-22    Northeast  New Hampshire  Manchester      Men's Street Footwear   
2021-02-22    Northeast  New Hampshire  Manchester    Men's Athletic Footwear   
2021-02-22    Northeast  New

In [8]:
data_cleaned = df.dropna()
data_cleaned.head(10)

,Region,State,City,Product,Price per Unit,Units Sold,Operating Margin
Invoice Date,,,,,,,
2020-01-01,Northeast,New York,New York,Men's Street Footwear,50.0,1200,0.50
2020-01-02,Northeast,New York,New York,Men's Athletic Footwear,50.0,1000,0.30
2020-01-03,Northeast,New York,New York,Women's Street Footwear,40.0,1000,0.35
2020-01-04,Northeast,New York,New York,Women's Athletic Footwear,45.0,850,0.35
2020-01-05,Northeast,New York,New York,Men's Apparel,60.0,900,0.30
2020-01-06,Northeast,New York,New York,Women's Apparel,50.0,1000,0.25
2020-01-07,Northeast,New York,New York,Men's Street Footwear,50.0,1250,0.50
2020-01-08,Northeast,New York,New York,Men's Athletic Footwear,50.0,900,0.30
2020-01-21,Northeast,New York,New York,Women's Street Footwear,40.0,950,0.35


In [9]:
data_cleaned['Total Sales'] = (data_cleaned['Price per Unit'] * data_cleaned['Units Sold']).astype(int)

data_cleaned.head(10)



,Region,State,City,Product,Price per Unit,Units Sold,Operating Margin,Total Sales
Invoice Date,,,,,,,,
2020-01-01,Northeast,New York,New York,Men's Street Footwear,50.0,1200,0.50,60000
2020-01-02,Northeast,New York,New York,Men's Athletic Footwear,50.0,1000,0.30,50000
2020-01-03,Northeast,New York,New York,Women's Street Footwear,40.0,1000,0.35,40000
2020-01-04,Northeast,New York,New York,Women's Athletic Footwear,45.0,850,0.35,38250
2020-01-05,Northeast,New York,New York,Men's Apparel,60.0,900,0.30,54000
2020-01-06,Northeast,New York,New York,Women's Apparel,50.0,1000,0.25,50000
2020-01-07,Northeast,New York,New York,Men's Street Footwear,50.0,1250,0.50,62500
2020-01-08,Northeast,New York,New York,Men's Athletic Footwear,50.0,900,0.30,45000
2020-01-21,Northeast,New York,New York,Women's Street Footwear,40.0,950,0.35,38000


In [10]:
data_cleaned['Total Operating Profit'] = (data_cleaned['Total Sales'] * data_cleaned['Operating Margin']).astype(float).dropna()

data_cleaned.head(10)


,Region,State,City,Product,Price per Unit,Units Sold,Operating Margin,Total Sales,Total Operating Profit
Invoice Date,,,,,,,,,
2020-01-01,Northeast,New York,New York,Men's Street Footwear,50.0,1200,0.50,60000,30000.00
2020-01-02,Northeast,New York,New York,Men's Athletic Footwear,50.0,1000,0.30,50000,15000.00
2020-01-03,Northeast,New York,New York,Women's Street Footwear,40.0,1000,0.35,40000,14000.00
2020-01-04,Northeast,New York,New York,Women's Athletic Footwear,45.0,850,0.35,38250,13387.50
2020-01-05,Northeast,New York,New York,Men's Apparel,60.0,900,0.30,54000,16200.00
2020-01-06,Northeast,New York,New York,Women's Apparel,50.0,1000,0.25,50000,12500.00
2020-01-07,Northeast,New York,New York,Men's Street Footwear,50.0,1250,0.50,62500,31250.00
2020-01-08,Northeast,New York,New York,Men's Athletic Footwear,50.0,900,0.30,45000,13500.00
2020-01-21,Northeast,New York,New York,Women's Street Footwear,40.0,950,0.35,38000,13300.00


In [11]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9648 entries, 2020-01-01 to 2021-02-22
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Region                  9648 non-null   object 
 1   State                   9648 non-null   object 
 2   City                    9648 non-null   object 
 3   Product                 9648 non-null   object 
 4   Price per Unit          9648 non-null   float64
 5   Units Sold              9648 non-null   int64  
 6   Operating Margin        9648 non-null   float64
 7   Total Sales             9648 non-null   int64  
 8   Total Operating Profit  9648 non-null   float64
dtypes: float64(3), int64(2), object(4)
memory usage: 753.8+ KB


In [12]:
data_cleaned['Profit per Unit'] = data_cleaned['Total Operating Profit'] / data_cleaned['Units Sold'].astype(float).dropna()
data_cleaned.head(10)


,Region,State,City,Product,Price per Unit,Units Sold,Operating Margin,Total Sales,Total Operating Profit,Profit per Unit
Invoice Date,,,,,,,,,,
2020-01-01,Northeast,New York,New York,Men's Street Footwear,50.0,1200,0.50,60000,30000.00,25.00
2020-01-02,Northeast,New York,New York,Men's Athletic Footwear,50.0,1000,0.30,50000,15000.00,15.00
2020-01-03,Northeast,New York,New York,Women's Street Footwear,40.0,1000,0.35,40000,14000.00,14.00
2020-01-04,Northeast,New York,New York,Women's Athletic Footwear,45.0,850,0.35,38250,13387.50,15.75
2020-01-05,Northeast,New York,New York,Men's Apparel,60.0,900,0.30,54000,16200.00,18.00
2020-01-06,Northeast,New York,New York,Women's Apparel,50.0,1000,0.25,50000,12500.00,12.50
2020-01-07,Northeast,New York,New York,Men's Street Footwear,50.0,1250,0.50,62500,31250.00,25.00
2020-01-08,Northeast,New York,New York,Men's Athletic Footwear,50.0,900,0.30,45000,13500.00,15.00
2020-01-21,Northeast,New York,New York,Women's Street Footwear,40.0,950,0.35,38000,13300.00,14.00


In [13]:


def subcat_assign(x):
    if "Wo" in x:  
        return "Women's"
    else:
        return "Men's"
    
data_cleaned['Subcategory'] = data_cleaned['Product'].apply(subcat_assign)



In [14]:
data_cleaned.tail(10)

,Region,State,City,Product,Price per Unit,Units Sold,Operating Margin,Total Sales,Total Operating Profit,Profit per Unit,Subcategory
Invoice Date,,,,,,,,,,,
2021-12-17,Northeast,New Hampshire,Manchester,Women's Apparel,61.0,144,0.40,8784,3513.60,24.40,Women's
2021-01-24,Northeast,New Hampshire,Manchester,Men's Street Footwear,36.0,165,0.37,5940,2197.80,13.32,Men's
2021-01-24,Northeast,New Hampshire,Manchester,Men's Athletic Footwear,36.0,93,0.40,3348,1339.20,14.40,Men's
2021-01-24,Northeast,New Hampshire,Manchester,Women's Street Footwear,35.0,99,0.27,3465,935.55,9.45,Women's
2021-01-24,Northeast,New Hampshire,Manchester,Women's Athletic Footwear,33.0,51,0.28,1683,471.24,9.24,Women's
2021-01-24,Northeast,New Hampshire,Manchester,Men's Apparel,50.0,64,0.28,3200,896.00,14.00,Men's
2021-01-24,Northeast,New Hampshire,Manchester,Women's Apparel,41.0,105,0.32,4305,1377.60,13.12,Women's
2021-02-22,Northeast,New Hampshire,Manchester,Men's Street Footwear,41.0,184,0.37,7544,2791.28,15.17,Men's
2021-02-22,Northeast,New Hampshire,Manchester,Men's Athletic Footwear,42.0,70,0.42,2940,1234.80,17.64,Men's


In [15]:
def cat_assign(x):
    if "Street" in x:  
        return "Street Footwear"
    if "Athletic" in x:
        return "Athletic Footwear"
    else:
        return "Apparel"
    
data_cleaned['Category'] = data_cleaned['Product'].apply(cat_assign)

data_cleaned.tail(10)

,Region,State,City,Product,Price per Unit,Units Sold,Operating Margin,Total Sales,Total Operating Profit,Profit per Unit,Subcategory,Category
Invoice Date,,,,,,,,,,,,
2021-12-17,Northeast,New Hampshire,Manchester,Women's Apparel,61.0,144,0.40,8784,3513.60,24.40,Women's,Apparel
2021-01-24,Northeast,New Hampshire,Manchester,Men's Street Footwear,36.0,165,0.37,5940,2197.80,13.32,Men's,Street Footwear
2021-01-24,Northeast,New Hampshire,Manchester,Men's Athletic Footwear,36.0,93,0.40,3348,1339.20,14.40,Men's,Athletic Footwear
2021-01-24,Northeast,New Hampshire,Manchester,Women's Street Footwear,35.0,99,0.27,3465,935.55,9.45,Women's,Street Footwear
2021-01-24,Northeast,New Hampshire,Manchester,Women's Athletic Footwear,33.0,51,0.28,1683,471.24,9.24,Women's,Athletic Footwear
2021-01-24,Northeast,New Hampshire,Manchester,Men's Apparel,50.0,64,0.28,3200,896.00,14.00,Men's,Apparel
2021-01-24,Northeast,New Hampshire,Manchester,Women's Apparel,41.0,105,0.32,4305,1377.60,13.12,Women's,Apparel
2021-02-22,Northeast,New Hampshire,Manchester,Men's Street Footwear,41.0,184,0.37,7544,2791.28,15.17,Men's,Street Footwear
2021-02-22,Northeast,New Hampshire,Manchester,Men's Athletic Footwear,42.0,70,0.42,2940,1234.80,17.64,Men's,Athletic Footwear


In [16]:
data_cleaned.columns

Index(['Region', 'State', 'City', 'Product', 'Price per Unit', 'Units Sold',
       'Operating Margin', 'Total Sales', 'Total Operating Profit',
       'Profit per Unit', 'Subcategory', 'Category'],
      dtype='object')

In [17]:
data_cleaned.reset_index(inplace=True)
print(data_cleaned.head())

  Invoice Date     Region     State      City                    Product  \
0   2020-01-01  Northeast  New York  New York      Men's Street Footwear   
1   2020-01-02  Northeast  New York  New York    Men's Athletic Footwear   
2   2020-01-03  Northeast  New York  New York    Women's Street Footwear   
3   2020-01-04  Northeast  New York  New York  Women's Athletic Footwear   
4   2020-01-05  Northeast  New York  New York              Men's Apparel   

   Price per Unit  Units Sold  Operating Margin  Total Sales  \
0            50.0        1200              0.50        60000   
1            50.0        1000              0.30        50000   
2            40.0        1000              0.35        40000   
3            45.0         850              0.35        38250   
4            60.0         900              0.30        54000   

   Total Operating Profit  Profit per Unit Subcategory           Category  
0                 30000.0            25.00       Men's    Street Footwear  
1     

In [18]:
data_cleaned['Invoice Date'] = pd.to_datetime(data_cleaned['Invoice Date'], errors='coerce')

In [19]:
data_cleaned['Month_Name'] = data_cleaned['Invoice Date'].dt.strftime('%b')
print(data_cleaned.tail())

     Invoice Date     Region          State        City  \
9643   2021-01-24  Northeast  New Hampshire  Manchester   
9644   2021-01-24  Northeast  New Hampshire  Manchester   
9645   2021-02-22  Northeast  New Hampshire  Manchester   
9646   2021-02-22  Northeast  New Hampshire  Manchester   
9647   2021-02-22  Northeast  New Hampshire  Manchester   

                      Product  Price per Unit  Units Sold  Operating Margin  \
9643            Men's Apparel            50.0          64              0.28   
9644          Women's Apparel            41.0         105              0.32   
9645    Men's Street Footwear            41.0         184              0.37   
9646  Men's Athletic Footwear            42.0          70              0.42   
9647  Women's Street Footwear            29.0          83              0.27   

      Total Sales  Total Operating Profit  Profit per Unit Subcategory  \
9643         3200                  896.00            14.00       Men's   
9644         4305     

In [20]:
data_cleaned['Year'] = data_cleaned['Invoice Date'].dt.year
print(data_cleaned.head())

  Invoice Date     Region     State      City                    Product  \
0   2020-01-01  Northeast  New York  New York      Men's Street Footwear   
1   2020-01-02  Northeast  New York  New York    Men's Athletic Footwear   
2   2020-01-03  Northeast  New York  New York    Women's Street Footwear   
3   2020-01-04  Northeast  New York  New York  Women's Athletic Footwear   
4   2020-01-05  Northeast  New York  New York              Men's Apparel   

   Price per Unit  Units Sold  Operating Margin  Total Sales  \
0            50.0        1200              0.50        60000   
1            50.0        1000              0.30        50000   
2            40.0        1000              0.35        40000   
3            45.0         850              0.35        38250   
4            60.0         900              0.30        54000   

   Total Operating Profit  Profit per Unit Subcategory           Category  \
0                 30000.0            25.00       Men's    Street Footwear   
1   

In [21]:
new_order = ['Invoice Date', 'Year', 'Month_Name', 'Region', 'State', 'City', 'Category', 'Subcategory', 'Product', 'Price per Unit', 'Units Sold', 'Total Sales', 'Profit per Unit', 'Operating Margin', 'Total Operating Profit'] 
data_cleaned = data_cleaned[new_order]
print(data_cleaned.head())

  Invoice Date  Year Month_Name     Region     State      City  \
0   2020-01-01  2020        Jan  Northeast  New York  New York   
1   2020-01-02  2020        Jan  Northeast  New York  New York   
2   2020-01-03  2020        Jan  Northeast  New York  New York   
3   2020-01-04  2020        Jan  Northeast  New York  New York   
4   2020-01-05  2020        Jan  Northeast  New York  New York   

            Category Subcategory                    Product  Price per Unit  \
0    Street Footwear       Men's      Men's Street Footwear            50.0   
1  Athletic Footwear       Men's    Men's Athletic Footwear            50.0   
2    Street Footwear     Women's    Women's Street Footwear            40.0   
3  Athletic Footwear     Women's  Women's Athletic Footwear            45.0   
4            Apparel       Men's              Men's Apparel            60.0   

   Units Sold  Total Sales  Profit per Unit  Operating Margin  \
0        1200        60000            25.00              0.50  

In [22]:
data_cleaned['Month_Number'] = data_cleaned['Invoice Date'].dt.month
print(data_cleaned.head())

  Invoice Date  Year Month_Name     Region     State      City  \
0   2020-01-01  2020        Jan  Northeast  New York  New York   
1   2020-01-02  2020        Jan  Northeast  New York  New York   
2   2020-01-03  2020        Jan  Northeast  New York  New York   
3   2020-01-04  2020        Jan  Northeast  New York  New York   
4   2020-01-05  2020        Jan  Northeast  New York  New York   

            Category Subcategory                    Product  Price per Unit  \
0    Street Footwear       Men's      Men's Street Footwear            50.0   
1  Athletic Footwear       Men's    Men's Athletic Footwear            50.0   
2    Street Footwear     Women's    Women's Street Footwear            40.0   
3  Athletic Footwear     Women's  Women's Athletic Footwear            45.0   
4            Apparel       Men's              Men's Apparel            60.0   

   Units Sold  Total Sales  Profit per Unit  Operating Margin  \
0        1200        60000            25.00              0.50  

In [23]:
columns = list(data_cleaned.columns)
print(data_cleaned.head())

  Invoice Date  Year Month_Name     Region     State      City  \
0   2020-01-01  2020        Jan  Northeast  New York  New York   
1   2020-01-02  2020        Jan  Northeast  New York  New York   
2   2020-01-03  2020        Jan  Northeast  New York  New York   
3   2020-01-04  2020        Jan  Northeast  New York  New York   
4   2020-01-05  2020        Jan  Northeast  New York  New York   

            Category Subcategory                    Product  Price per Unit  \
0    Street Footwear       Men's      Men's Street Footwear            50.0   
1  Athletic Footwear       Men's    Men's Athletic Footwear            50.0   
2    Street Footwear     Women's    Women's Street Footwear            40.0   
3  Athletic Footwear     Women's  Women's Athletic Footwear            45.0   
4            Apparel       Men's              Men's Apparel            60.0   

   Units Sold  Total Sales  Profit per Unit  Operating Margin  \
0        1200        60000            25.00              0.50  

In [24]:
columns.remove('Month_Number')

In [25]:
month_name_index = columns.index('Month_Name')
columns.insert(month_name_index, 'Month_Number')
data_cleaned = data_cleaned[columns]
print(data_cleaned.head())

  Invoice Date  Year  Month_Number Month_Name     Region     State      City  \
0   2020-01-01  2020             1        Jan  Northeast  New York  New York   
1   2020-01-02  2020             1        Jan  Northeast  New York  New York   
2   2020-01-03  2020             1        Jan  Northeast  New York  New York   
3   2020-01-04  2020             1        Jan  Northeast  New York  New York   
4   2020-01-05  2020             1        Jan  Northeast  New York  New York   

            Category Subcategory                    Product  Price per Unit  \
0    Street Footwear       Men's      Men's Street Footwear            50.0   
1  Athletic Footwear       Men's    Men's Athletic Footwear            50.0   
2    Street Footwear     Women's    Women's Street Footwear            40.0   
3  Athletic Footwear     Women's  Women's Athletic Footwear            45.0   
4            Apparel       Men's              Men's Apparel            60.0   

   Units Sold  Total Sales  Profit per Unit 

In [26]:
data_cleaned = data_cleaned.loc[:, ~data_cleaned.columns.duplicated()]
print(data_cleaned.head())


  Invoice Date  Year  Month_Number Month_Name     Region     State      City  \
0   2020-01-01  2020             1        Jan  Northeast  New York  New York   
1   2020-01-02  2020             1        Jan  Northeast  New York  New York   
2   2020-01-03  2020             1        Jan  Northeast  New York  New York   
3   2020-01-04  2020             1        Jan  Northeast  New York  New York   
4   2020-01-05  2020             1        Jan  Northeast  New York  New York   

            Category Subcategory                    Product  Price per Unit  \
0    Street Footwear       Men's      Men's Street Footwear            50.0   
1  Athletic Footwear       Men's    Men's Athletic Footwear            50.0   
2    Street Footwear     Women's    Women's Street Footwear            40.0   
3  Athletic Footwear     Women's  Women's Athletic Footwear            45.0   
4            Apparel       Men's              Men's Apparel            60.0   

   Units Sold  Total Sales  Profit per Unit 

In [27]:
data_cleaned['Year-Month'] = data_cleaned['Invoice Date'].dt.to_period('M').astype(str)
print(data_cleaned.head())


  Invoice Date  Year  Month_Number Month_Name     Region     State      City  \
0   2020-01-01  2020             1        Jan  Northeast  New York  New York   
1   2020-01-02  2020             1        Jan  Northeast  New York  New York   
2   2020-01-03  2020             1        Jan  Northeast  New York  New York   
3   2020-01-04  2020             1        Jan  Northeast  New York  New York   
4   2020-01-05  2020             1        Jan  Northeast  New York  New York   

            Category Subcategory                    Product  Price per Unit  \
0    Street Footwear       Men's      Men's Street Footwear            50.0   
1  Athletic Footwear       Men's    Men's Athletic Footwear            50.0   
2    Street Footwear     Women's    Women's Street Footwear            40.0   
3  Athletic Footwear     Women's  Women's Athletic Footwear            45.0   
4            Apparel       Men's              Men's Apparel            60.0   

   Units Sold  Total Sales  Profit per Unit 

In [28]:
grouped_data = data_cleaned.groupby(
    ['Year-Month', 'Category', 'Subcategory', 'State', 'Region']
).sum(numeric_only=True).reset_index()
print(data_cleaned.head())

  Invoice Date  Year  Month_Number Month_Name     Region     State      City  \
0   2020-01-01  2020             1        Jan  Northeast  New York  New York   
1   2020-01-02  2020             1        Jan  Northeast  New York  New York   
2   2020-01-03  2020             1        Jan  Northeast  New York  New York   
3   2020-01-04  2020             1        Jan  Northeast  New York  New York   
4   2020-01-05  2020             1        Jan  Northeast  New York  New York   

            Category Subcategory                    Product  Price per Unit  \
0    Street Footwear       Men's      Men's Street Footwear            50.0   
1  Athletic Footwear       Men's    Men's Athletic Footwear            50.0   
2    Street Footwear     Women's    Women's Street Footwear            40.0   
3  Athletic Footwear     Women's  Women's Athletic Footwear            45.0   
4            Apparel       Men's              Men's Apparel            60.0   

   Units Sold  Total Sales  Profit per Unit 

In [31]:

print(grouped_data)

     Year-Month         Category Subcategory          State     Region   Year  \
0       2020-01          Apparel       Men's         Nevada       West  30300   
1       2020-01          Apparel       Men's       New York  Northeast  18180   
2       2020-01          Apparel     Women's         Nevada       West  30300   
3       2020-01          Apparel     Women's       New York  Northeast  18180   
4       2020-01          Apparel     Women's   Pennsylvania  Northeast   6060   
...         ...              ...         ...            ...        ...    ...   
3025    2021-12  Street Footwear     Women's        Vermont  Northeast   6063   
3026    2021-12  Street Footwear     Women's       Virginia  Southeast   6063   
3027    2021-12  Street Footwear     Women's  West Virginia  Northeast   4042   
3028    2021-12  Street Footwear     Women's      Wisconsin    Midwest   4042   
3029    2021-12  Street Footwear     Women's        Wyoming       West   4042   

      Month_Number  Price p

In [35]:
new_order = ['Invoice Date', 'Year-Month','Category', 'Subcategory', 'Region', 'State', 'City','Price per Unit', 'Units Sold', 'Total Sales', 'Operating Margin', 'Profit per Unit', "Total Operating Profit", 'Year', 'Month_Name', 'Month_Number', 'Product'] 
data_cleaned = data_cleaned[new_order]
print(data_cleaned.head())

  Invoice Date Year-Month           Category Subcategory     Region     State  \
0   2020-01-01    2020-01    Street Footwear       Men's  Northeast  New York   
1   2020-01-02    2020-01  Athletic Footwear       Men's  Northeast  New York   
2   2020-01-03    2020-01    Street Footwear     Women's  Northeast  New York   
3   2020-01-04    2020-01  Athletic Footwear     Women's  Northeast  New York   
4   2020-01-05    2020-01            Apparel       Men's  Northeast  New York   

       City  Price per Unit  Units Sold  Total Sales  Operating Margin  \
0  New York            50.0        1200        60000              0.50   
1  New York            50.0        1000        50000              0.30   
2  New York            40.0        1000        40000              0.35   
3  New York            45.0         850        38250              0.35   
4  New York            60.0         900        54000              0.30   

   Profit per Unit  Total Operating Profit  Year Month_Name  Month_N

In [36]:
grouped_data.to_csv('Adidas_Sales_Data_Monthly_Category_Summary.csv', index=False)
